# Model - YOLOv8
# type - One vs All
# Dataset - extended
# Test - VisDrone (classwise)

# 1. Install Dependencies


In [1]:
!git clone https://github.com/mohitsharma-iitj/SEblock_yolov8.git
%cd SEblock_yolov8
!pip install -r requirements.txt


Cloning into 'SEblock_yolov8'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 205 (delta 36), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (205/205), 3.09 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/kaggle/working/SEblock_yolov8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 MB 19.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 11.0 MB/s eta

# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image


# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if len(annotation) != 8:
#                 print(f"Error parsing annotation in {filename}: {annotation}")
#                 print(f"Ignoring malformed annotation: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            center_x = x_min + (width / 2.0)
            center_y = y_min + (height / 2.0)
            center_x /= img_width
            center_y /= img_height
            width /= img_width
            height /= img_height
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


### train,val from visdrone

In [3]:
def create_files_train(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images'
    os.makedirs( '/kaggle/working/yolo_dataset', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset')

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'labels/train')
    output_images_path = os.path.join(output_dataset_path, 'images/train')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [4]:
def create_files_val(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images'

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'labels/val')
    output_images_path = os.path.join(output_dataset_path, 'images/val')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

## for test from expanded

In [5]:
def create_files_test(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/expanded-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/expanded-dataset/Expanded_dataset/images'

    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'labels/test')
    output_images_path = os.path.join(output_dataset_path, 'images/test')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [6]:
def classes_label(class_name):
    create_files_train(class_name)
    create_files_val(class_name)
    create_files_test(class_name)

# 3. Import yolov8

In [7]:
import wandb
from ultralytics import YOLO


# 4. Train Class wise



## Class 6 - tricycle

In [8]:
classes_label(6)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)


In [9]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  
    
# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='tricycle_1')




WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [10]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


del model

from IPython.display import clear_output
clear_output(wait=True)

import gc
gc.collect()

376571

In [11]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1301 images, 1133 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1858.02it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:14<00:00,  5.56it/s]
                   all       1301        338   0.000279       0.29   0.000508   0.000153
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pre

## Class 7 - awning-tricycle

In [12]:
# Remove the directory
shutil.rmtree('/kaggle/working/yolo_dataset')

classes_label(7)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['awning-tricycle']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [13]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
 
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='awning-tricycle_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [14]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

del model

from IPython.display import clear_output
clear_output(wait=True)

import gc
gc.collect()

714037

In [15]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/awning-tricycle_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1301 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:02<00:00, 572.45it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:15<00:00,  5.14it/s]
                   all       1301        390   5.12e-06    0.00513   2.64e-06   5.22e-07
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 3.4ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pred

## Class 8 - bus

In [9]:
# Remove the directory
shutil.rmtree('/kaggle/working/yolo_dataset')

classes_label(8)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['bus']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [10]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='bus_1')


WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [11]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

del model

from IPython.display import clear_output
clear_output(wait=True)

import gc
gc.collect()

84129

In [12]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/bus_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1301 images, 718 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1503.50it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:15<00:00,  5.24it/s]
                   all       1301       1640      0.446      0.327      0.286      0.172
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pred

## Class 9 - motor

In [8]:
# Remove the directory
shutil.rmtree('/kaggle/working/yolo_dataset')

classes_label(9)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['motor']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)




In [9]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='motor_1')




WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [10]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

del model

from IPython.display import clear_output
clear_output(wait=True)

import gc
gc.collect()

270140

In [11]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/motor_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1301 images, 724 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:00<00:00, 1585.82it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:14<00:00,  5.72it/s]
                   all       1301       3944     0.0683     0.0322     0.0147    0.00477
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Saving /kaggle/working/SEblock_yolov8/runs/detect/val/pred

## Class 10- others

In [12]:
# Remove the directory
shutil.rmtree('/kaggle/working/yolo_dataset')

classes_label(10)


data_config = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val
test: /kaggle/working/yolo_dataset/images/test

nc: 1  # number of classes 
names: ['others']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)


In [13]:
!export WANDB_MODE=disabled
wandb.init(mode="disabled")
    
model = YOLO('/kaggle/working/SEblock_yolov8/ultralytics/cfg/models/v8/yolov8_SEModule.yaml').load('/kaggle/working/SEblock_yolov8/yolov8n.pt')  

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=25, imgsz=640, batch=16, name='others_1')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultral

In [14]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')

del model

from IPython.display import clear_output
clear_output(wait=True)

import gc
gc.collect()

273695

In [15]:
# Load the best model
model = YOLO('/kaggle/working/SEblock_yolov8/runs/detect/others_1/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.0.147 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_SEModule summary (fused): 188 layers, 3007575 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/yolo_dataset/labels/test... 1301 images, 1196 backgrounds, 0 corrupt: 100%|██████████| 1301/1301 [00:02<00:00, 520.22it/s]
val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:10<00:00,  7.46it/s]
                   all       1301        171          0          0          0          0
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /kaggle/working/SEblock_yolov8/runs/detec